In [1]:
%load_ext autoreload
%autoreload 2
!pwd
import os
import sys 
if os.path.abspath("../../../") not in sys.path:
    sys.path.append(os.path.abspath("../../../")) 

import os
import networkx as nx
from rwtools.nemesis.graph.nemesis_node import AbstractNemesisNode
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg
from rwtools.nemesis.control_flow_graph import ControlFlowGraph


/home/gilles/git-repos/NemesisRetroWrite/retrowrite/rwtools/nemesis/samples_notebooks


In [2]:
import sys 
sys.executable

'/home/gilles/git-repos/NemesisRetroWrite/retrowrite/venv/bin/python'

In [3]:
from test_main import setup 

binary = os.path.abspath("./password_comparison/password_comparison")
outputfile = "./inserted_nodes.s"
loader, rw = setup(binary, outputfile)
rw.dump()

[*] Relocations for a section that's not loaded: .rela.dyn
[x] Could not replace value in .init_array
[x] Couldn't find valid section 3e20
[x] Couldn't find valid section 3fd8
[x] Couldn't find valid section 3fe0
[x] Couldn't find valid section 3fe8
[x] Couldn't find valid section 3ff0
[x] Couldn't find valid section 3ff8


In [4]:

outputfile = "./balancing_test.s"
loader, rw = setup(binary, outputfile)

# function_names = loader.container.function_names



nodes, graph = create_graph_structure(loader.container, "main")
fig, axs = plt.subplots(1, figsize=(30, 30))

cfg = ControlFlowGraph(nodes=nodes, graph=graph)

cfg.merge_consecutive_nodes()

cfg.unwind_graph()

# insert nodes 
cfg.insert_nodes()

cfg.equalize_branches()



RegisterAnalysis.analyze(loader.container)

main_function = None
for addr, function in loader.container.functions.items():
    if function.name == "main":
        main_function = function 
        
free_registers = main_function.analysis['free_registers']
clobber_registers = list(main_function.analysis['clobber_registers'])

# convert these into format the balancing algorithm can use 
new_free_registrs = {}
for idx, wrapper in enumerate(function.cache):
    new_free_registrs[function.cache[idx]] = free_registers[idx]
    
def get_nop_v2(target_lat):
    if target_lat == 1: 
        return "movq {}, {}", 2
    elif target_lat == 3: 
        return "imul {}, {}", 2
    else: 
        return "imul {}, {}", 2

    
def balance_nodeset(nodes):
    print(f"balancing nodes: {[node.id for node in nodes]}")
    it = 0
    while True: 
        # map each node to its length 
        longest = max(nodes, key=lambda n: n.num_instructions())
        if it >= longest.num_instructions():
            break
        target_lat = longest.get_instr_latency(it)
        dummy_instruction, n_args = get_nop_v2(target_lat)
        if dummy_instruction == "placeholder":
            it += 1
            continue 
        for node in [n for n in nodes if n != longest]:
            reg = n_args*[f"%{clobber_registers[0]}"]
            dummy_instruction = dummy_instruction.format(*reg)
            node.insert(it, dummy_instruction, target_lat)
        it +=1 
        
def balancing_algorithm():
    root = cfg.get_node('1130')
    tree_depths = nx.shortest_path_length(cfg.graph, root)
    for i in range(max(tree_depths.values())+1):
        level_nodes = [node for node in cfg.graph if tree_depths[node] == i]
        if len(level_nodes) <= 1: 
            continue 
        else: 
            balance_nodeset(level_nodes)
            
balancing_algorithm()
    

cfg.merge_inserted_nodes()
axs.imshow(cfg.to_img())

    
rw.dump()

[*] Relocations for a section that's not loaded: .rela.dyn
[x] Could not replace value in .init_array
[x] Couldn't find valid section 3e20
[x] Couldn't find valid section 3fd8
[x] Couldn't find valid section 3fe0
[x] Couldn't find valid section 3fe8
[x] Couldn't find valid section 3ff0
[x] Couldn't find valid section 3ff8


NameError: name 'create_graph_structure' is not defined